Se tomara como base https://www.kaggle.com/joanfo/word2vec-cnn-extra-features-0-9757

In [1]:
import pandas as pd
from   keras                        import initializers, regularizers, constraints, callbacks, optimizers
from   keras.layers                 import Conv1D, Embedding, GlobalMaxPooling1D, concatenate, Input, Dense
from   keras.models                 import Sequential, Model
from   keras.preprocessing.sequence import pad_sequences
from   keras.preprocessing.text     import Tokenizer
from   sklearn.model_selection      import train_test_split
from   gensim.models.keyedvectors   import KeyedVectors
import numpy as np

Traigo Los Datos

In [2]:
from google.colab import files
import io

In [3]:
uploaded = files.upload()

Saving test.csv to test.csv
Saving twiter_with_data_location_v2.csv to twiter_with_data_location_v2.csv


In [4]:
#tw = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))
tw_test = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))
#twiter_my=pd.read_csv(io.StringIO(uploaded['twiter_with_data_location.csv'].decode('utf-8')))
twiter_my2=pd.read_csv(io.StringIO(uploaded['twiter_with_data_location_v2.csv'].decode('utf-8')))

In [5]:
twiter_my2.head(2)

,id,keyword,location,text,target,location_isnull,location_free,address_latitude_longitude,latitude,longitude,city,country
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,True,NaN,NaN,NaN,NaN,NaN,NaN
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,True,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
twiter_my2["len_text"]=twiter_my2.text.apply(len)

In [7]:
twiter_my2.len_text.head(2)

0    69
1    38
Name: len_text, dtype: int64

In [8]:
def countWords(words):
  words=re.split(r'\W+', words)
  cont=0
  unique_words=[]
  for word in words:
    if (word not in unique_words):
      cont=cont+1
      unique_words.append(word)
  return cont

In [9]:
import re

In [10]:
twiter_my2["count_word_uniques"]=twiter_my2.text.apply(lambda x: countWords(x))

In [11]:
twiter_my2.count_word_uniques

0       13
1        7
2       18
3       10
4       16
        ..
7608    14
7609    18
7610    14
7611    22
7612    15
Name: count_word_uniques, Length: 7613, dtype: int64

In [12]:
tw_test["len_text"]=tw_test.text.apply(len)
tw_test["count_word_uniques"]=tw_test.text.apply(lambda x: countWords(x))

In [13]:
train=twiter_my2.loc[:, ['id',"text", 'len_text', 'count_word_uniques']]  
test=   tw_test.loc[:, ['id',"text", 'len_text', 'count_word_uniques']]  

In [14]:
total_words = train.loc[:, 'id' : 'text']
total_words=pd.concat([total_words,test], sort = 'False')


In [15]:
total_words.head(2)

,count_word_uniques,id,len_text,text
0,NaN,1,NaN,Our Deeds are the Reason of this #earthquake M...
1,NaN,4,NaN,Forest fire near La Ronge Sask. Canada


**Comenzando a Tokenizar**

In [16]:
numWords=20000
tokenizer = Tokenizer(num_words = numWords)

In [17]:
tokenizer.fit_on_texts(total_words['text'])

In [18]:
sequences_train  = tokenizer.texts_to_sequences(train['text'])
sequences_test   = tokenizer.texts_to_sequences(test['text'])

In [19]:
word_index       = tokenizer.word_index

In [20]:
#max_length       = 200
max_length       = 300
x_train          = pad_sequences(sequences_train, maxlen = max_length)
x_test           = pad_sequences(sequences_test, maxlen = max_length)
x_features_train = train[['len_text', 'count_word_uniques']]
x_features_test  = test[['len_text', 'count_word_uniques']]
y_train          = twiter_my2["target"].values

Cargo Word2Vect

In [21]:
#word_vectors se obtiene de word2vec, que asigna casi todas las palabras en inglés a un tamaño de 300
#vectores que tiene una corta distancia cos con palabras cercanas cercanas
word_vectors = KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary = True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Comienzo la creacion de mi matriz embedign**

In [22]:
#pongo cada palabra en word_index para tener una matriz donde cada fila i contendra el correspondiente vector obtenido del Word2vect
#embeddind_dimension = 300
embeddind_dimension = 300
vocabulary_size = min(len(word_index) + 1, numWords)
embedding_matrix = np.zeros((vocabulary_size, embeddind_dimension))

In [23]:
for word, i in word_index.items():
    if i < numWords:
        try:
            embedding_vector = word_vectors[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), embeddind_dimension)

del(word_vectors)

**Comienzo a crear mi modelo**

In [24]:
input_embedding = Input(shape = (max_length,))

In [25]:
embedding = Embedding(vocabulary_size,
                      embeddind_dimension,
                      weights = [embedding_matrix],
                      trainable = False)(input_embedding)

**Usando un modelo Secuencial**

In [26]:
e = Embedding(len(embedding_matrix), 200, input_length=300)

In [27]:
from keras.layers import Dropout

In [28]:
model1 = Sequential()
model1.add(e)
model1.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model1.add(GlobalMaxPooling1D())
model1.add(Dropout(0.25))
model1.add(Dense(256, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d (Conv1D)              (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 4,066,213
Trainable params: 4,066,213
Non-trainable params: 0
______________________________________________

In [29]:
Xtrain1, Xval1, ytrain1, yval1 = train_test_split(x_train, y_train,
                                              train_size = 0.95, random_state = 233)

Xfeaturestrain1, Xfeaturesval1, ytrain1, yval1 = train_test_split(x_features_train, y_train,
                                                            train_size = 0.95, random_state = 233)

In [30]:
model1.fit(Xtrain1, ytrain1,
          validation_data=(Xval1, yval1), 
          epochs=5, 
          batch_size=300,
          verbose=1)

Epoch 1/5
25/25 [==============================] - 16s 629ms/step - loss: 0.6554 - accuracy: 0.6113 - val_loss: 0.5787 - val_accuracy: 0.7139
Epoch 2/5
25/25 [==============================] - 15s 617ms/step - loss: 0.4825 - accuracy: 0.7893 - val_loss: 0.4532 - val_accuracy: 0.7822
Epoch 3/5
25/25 [==============================] - 15s 617ms/step - loss: 0.2824 - accuracy: 0.8967 - val_loss: 0.4494 - val_accuracy: 0.8163
Epoch 4/5
25/25 [==============================] - 16s 620ms/step - loss: 0.1453 - accuracy: 0.9527 - val_loss: 0.5063 - val_accuracy: 0.8005
Epoch 5/5
25/25 [==============================] - 15s 617ms/step - loss: 0.0747 - accuracy: 0.9784 - val_loss: 0.5600 - val_accuracy: 0.8031


In [32]:
y_pred1 = model1.predict_classes([x_test, x_features_test])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [33]:
y_pred1

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [34]:
submit1 = tw_test['id'].to_frame()
submit1['target'] = pd.DataFrame(y_pred1)

In [35]:
submit1.to_csv('submit1.csv', index=False)
files.download('submit1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Usando un modelo Model**


In [38]:
conv1 = Conv1D(filters = 100, kernel_size = 2, padding = 'same')(embedding)
conv2 = Conv1D(filters = 100, kernel_size = 3, padding = 'same')(embedding)
conv3 = Conv1D(filters = 100, kernel_size = 4, padding = 'same')(embedding)

pool1 = GlobalMaxPooling1D()(conv1)
pool2 = GlobalMaxPooling1D()(conv2)
pool2 = GlobalMaxPooling1D()(conv3)

In [39]:
input_features = Input(shape = (2,))
merged_tensor = concatenate([pool1, pool2, pool2, input_features], axis = 1)
output = Dense(units = 1, activation = 'sigmoid')(merged_tensor)
model2 = Model(inputs = [input_embedding, input_features], outputs = output)

In [40]:
model2.compile(loss      = 'binary_crossentropy',
              optimizer = optimizers.Adam(),
              metrics   = ['accuracy'])

In [41]:
Xtrain, Xval, ytrain, yval = train_test_split(x_train, y_train,
                                              train_size = 0.95, random_state = 233)

Xfeaturestrain, Xfeaturesval, ytrain, yval = train_test_split(x_features_train, y_train,
                                                            train_size = 0.95, random_state = 233)

In [42]:
model2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 300, 300)     6000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 300, 100)     60100       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 300, 100)     120100      embedding[0][0]                  
_______________________________________________________________________________________

In [43]:
model2.fit(x = [Xtrain, Xfeaturestrain], y = ytrain, epochs=5,
          validation_data = ([Xval, Xfeaturesval], yval), verbose = 1)

Epoch 1/5
226/226 [==============================] - 45s 198ms/step - loss: 1.0822 - accuracy: 0.6050 - val_loss: 0.5961 - val_accuracy: 0.6955
Epoch 2/5
226/226 [==============================] - 45s 198ms/step - loss: 0.4256 - accuracy: 0.8092 - val_loss: 0.5094 - val_accuracy: 0.7533
Epoch 3/5
226/226 [==============================] - 44s 197ms/step - loss: 0.2944 - accuracy: 0.8906 - val_loss: 0.4980 - val_accuracy: 0.7848
Epoch 4/5
226/226 [==============================] - 44s 196ms/step - loss: 0.2058 - accuracy: 0.9354 - val_loss: 0.5389 - val_accuracy: 0.7585
Epoch 5/5
226/226 [==============================] - 44s 196ms/step - loss: 0.1432 - accuracy: 0.9562 - val_loss: 0.6206 - val_accuracy: 0.7664


In [44]:
y_pred2 = model2.predict([x_test, x_features_test])

In [45]:
#y_pred2

In [46]:
#import math

In [48]:
submit2 = tw_test['id'].to_frame()
submit2['target'] = pd.DataFrame(y_pred2)

In [49]:
#submit.target=submit.target.apply(lambda x: math.ceil(x))
submit2.target=submit2.target.apply(lambda x: round(x))
submit2

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [50]:
submit2.to_csv('submit2.csv', index=False)
files.download('submit2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
model3 = Sequential()
model3.add(e)
model3.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model3.add(GlobalMaxPooling1D())
model3.add(Dense(200, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(200, activation='relu'))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 100)              

In [52]:
model3.fit(x = [Xtrain, Xfeaturestrain], y = ytrain, epochs=5,
          validation_data = ([Xval, Xfeaturesval], yval), verbose = 1)

Epoch 1/5
226/226 [==============================] - 26s 114ms/step - loss: 0.2350 - accuracy: 0.9038 - val_loss: 0.5369 - val_accuracy: 0.7769
Epoch 2/5
226/226 [==============================] - 29s 129ms/step - loss: 0.0665 - accuracy: 0.9784 - val_loss: 0.7376 - val_accuracy: 0.7402
Epoch 3/5
226/226 [==============================] - 25s 113ms/step - loss: 0.0363 - accuracy: 0.9899 - val_loss: 0.7462 - val_accuracy: 0.7795
Epoch 4/5
226/226 [==============================] - 25s 113ms/step - loss: 0.0249 - accuracy: 0.9916 - val_loss: 0.8793 - val_accuracy: 0.7087
Epoch 5/5
226/226 [==============================] - 26s 113ms/step - loss: 0.0147 - accuracy: 0.9923 - val_loss: 0.9364 - val_accuracy: 0.7559


In [53]:

y_pred3 = model3.predict_classes([x_test, x_features_test])

In [54]:
y_pred3

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [55]:
submit3 = tw_test['id'].to_frame()
submit3['target'] = pd.DataFrame(y_pred3)

In [56]:
submit3.to_csv('submit3.csv', index=False)
files.download('submit3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
model4 = Sequential()
model4.add(e)
model4.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model4.add(GlobalMaxPooling1D())
model4.add(Dense(200, activation='relu'))
model4.add(Dropout(0.25))
model4.add(Dense(200, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(50, activation='relu'))
model4.add(Dropout(0.25))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_9 (Dense)              (None, 100)              

In [58]:
model4.fit(x = [Xtrain, Xfeaturestrain], y = ytrain, epochs=5,
          validation_data = ([Xval, Xfeaturesval], yval), verbose = 1)

Epoch 1/5
226/226 [==============================] - 26s 114ms/step - loss: 0.1127 - accuracy: 0.9605 - val_loss: 0.9151 - val_accuracy: 0.7270
Epoch 2/5
226/226 [==============================] - 26s 114ms/step - loss: 0.0367 - accuracy: 0.9884 - val_loss: 0.8986 - val_accuracy: 0.7323
Epoch 3/5
226/226 [==============================] - 26s 115ms/step - loss: 0.0234 - accuracy: 0.9920 - val_loss: 1.3987 - val_accuracy: 0.7664
Epoch 4/5
226/226 [==============================] - 26s 113ms/step - loss: 0.0150 - accuracy: 0.9927 - val_loss: 1.3847 - val_accuracy: 0.7638
Epoch 5/5
226/226 [==============================] - 26s 114ms/step - loss: 0.0137 - accuracy: 0.9928 - val_loss: 1.7960 - val_accuracy: 0.7559


In [59]:
y_pred4 = model4.predict_classes([x_test, x_features_test])

In [60]:
submit4 = tw_test['id'].to_frame()
submit4['target'] = pd.DataFrame(y_pred4)

In [61]:
submit4

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [62]:
submit4.to_csv('submit4.csv', index=False)
files.download('submit4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
model5 = Sequential()
model5.add(e)
model5.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model5.add(GlobalMaxPooling1D())
model5.add(Dense(200, activation='relu'))
model5.add(Dropout(0.25))
model5.add(Dense(200, activation='relu'))
model5.add(Dense(50, activation='relu'))
model5.add(Dropout(0.5))
model5.add(Dense(1, activation='sigmoid'))
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 200)               20200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_14 (Dense)             (None, 50)               

In [66]:
model5.fit(x = [Xtrain, Xfeaturestrain], y = ytrain, epochs=5,
          validation_data = ([Xval, Xfeaturesval], yval), verbose = 1)

Epoch 1/5
226/226 [==============================] - 26s 113ms/step - loss: 0.0863 - accuracy: 0.9729 - val_loss: 1.0917 - val_accuracy: 0.7349
Epoch 2/5
226/226 [==============================] - 25s 113ms/step - loss: 0.0251 - accuracy: 0.9920 - val_loss: 1.0568 - val_accuracy: 0.7533
Epoch 3/5
226/226 [==============================] - 26s 113ms/step - loss: 0.0152 - accuracy: 0.9932 - val_loss: 2.1594 - val_accuracy: 0.7638
Epoch 4/5
226/226 [==============================] - 26s 113ms/step - loss: 0.0122 - accuracy: 0.9934 - val_loss: 2.0030 - val_accuracy: 0.7533
Epoch 5/5
226/226 [==============================] - 26s 113ms/step - loss: 0.0201 - accuracy: 0.9909 - val_loss: 1.8709 - val_accuracy: 0.7480


In [68]:
y_pred5 = model5.predict_classes([x_test, x_features_test])

In [70]:
submit5 = tw_test['id'].to_frame()
submit5['target'] = pd.DataFrame(y_pred5)

In [71]:
submit5.to_csv('submit5.csv', index=False)
files.download('submit5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [77]:
uploaded2 = files.upload()

Saving submit_kaggle_combinate.csv to submit_kaggle_combinate.csv
Saving submit_kaggle_false.csv to submit_kaggle_false (1).csv
Saving submit_kaggle_true.csv to submit_kaggle_true (1).csv
Saving submit_kaggle_true_false.csv to submit_kaggle_true_false (1).csv


In [78]:
submit_kaggle_false = pd.read_csv(io.StringIO(uploaded2['submit_kaggle_false.csv'].decode('utf-8')))
submit_kaggle_true = pd.read_csv(io.StringIO(uploaded2['submit_kaggle_true.csv'].decode('utf-8')))
submit_kaggle_true_false = pd.read_csv(io.StringIO(uploaded2['submit_kaggle_true_false.csv'].decode('utf-8')))
submit_kaggle_combinate = pd.read_csv(io.StringIO(uploaded2['submit_kaggle_combinate.csv'].decode('utf-8')))

In [91]:
uploaded3 = files.upload()

Saving submit_kaggle.csv to submit_kaggle.csv
Saving submit_kaggle_combinate.csv to submit_kaggle_combinate (1).csv
Saving submit_kaggle2.csv to submit_kaggle2.csv


In [93]:
submit_kaggle_combinate1 = pd.read_csv(io.StringIO(uploaded3['submit_kaggle.csv'].decode('utf-8')))
submit_kaggle_combinate2 = pd.read_csv(io.StringIO(uploaded3['submit_kaggle_combinate.csv'].decode('utf-8')))
submit_kaggle_combinate3 = pd.read_csv(io.StringIO(uploaded3['submit_kaggle2.csv'].decode('utf-8')))

USANDO RESULTADOS PRINCIPALES

In [98]:
uploaded4 = files.upload()

Saving submit_kaggle_2.csv to submit_kaggle_2.csv
Saving submit_kaggle_4.csv to submit_kaggle_4.csv
Saving submit_kaggle_5.csv to submit_kaggle_5.csv
Saving submit_kaggle_Combinate_6.csv to submit_kaggle_Combinate_6.csv
Saving Sumbit_Kaggle_1.csv to Sumbit_Kaggle_1.csv
Saving submit1.csv to submit1 (1).csv
Saving submit5.csv to submit5 (1).csv
Saving submit4 (1).csv to submit4 (1).csv
Saving submit3 (2).csv to submit3 (2).csv
Saving submit2.csv to submit2 (1).csv


In [103]:
submit_kaggle_2 = pd.read_csv(io.StringIO(uploaded4['submit_kaggle_2.csv'].decode('utf-8')))
submit_kaggle_4 = pd.read_csv(io.StringIO(uploaded4['submit_kaggle_4.csv'].decode('utf-8')))
submit_kaggle_5 = pd.read_csv(io.StringIO(uploaded4['submit_kaggle_5.csv'].decode('utf-8')))
submit_kaggle_Combinate_6 = pd.read_csv(io.StringIO(uploaded4['submit_kaggle_Combinate_6.csv'].decode('utf-8')))
Sumbit_Kaggle_1 = pd.read_csv(io.StringIO(uploaded4['Sumbit_Kaggle_1.csv'].decode('utf-8')))
submit1 = pd.read_csv(io.StringIO(uploaded4['submit1.csv'].decode('utf-8')))
submit2 = pd.read_csv(io.StringIO(uploaded4['submit2.csv'].decode('utf-8')))
submit3 = pd.read_csv(io.StringIO(uploaded4['submit3 (2).csv'].decode('utf-8')))
submit4= pd.read_csv(io.StringIO(uploaded4['submit4 (1).csv'].decode('utf-8')))
submit5 = pd.read_csv(io.StringIO(uploaded4['submit5.csv'].decode('utf-8')))

In [80]:
submitFinal = tw_test['id'].to_frame()

In [108]:
submitFinal["pred1"]=submit_kaggle_2.target
submitFinal["pred2"]=submit_kaggle_4.target
submitFinal["pred3"]=submit_kaggle_5.target
submitFinal["pred4"]=submit_kaggle_Combinate_6.target
submitFinal["pred5"]=Sumbit_Kaggle_1.target
submitFinal["pred6"]=submit1.target
submitFinal["pred7"]=submit2.target
submitFinal["pred8"]=submit3.target
submitFinal["pred9"]=submit4.target
#submitFinal["pred10"]=submit5.target

In [86]:
#submitFinal["pred1"]=submit1.target
#submitFinal["pred2"]=submit2.target
#submitFinal["pred3"]=submit3.target
#submitFinal["pred4"]=submit4.target
#submitFinal["pred5"]=submit5.target
#submitFinal["pred6"]=submit_kaggle_false.target
#submitFinal["pred7"]=submit_kaggle_true.target
#submitFinal["pred8"]=submit_kaggle_true_false.target
#submitFinal["pred9"]=submit_kaggle_combinate.target
#submitFinal["pred10"]=submit_kaggle_combinate1.target
#submitFinal["pred11"]=submit_kaggle_combinate2.target
#submitFinal["pred12"]=submit_kaggle_combinate3.target

In [109]:
submitFinal

,id,pred1,pred2,pred3,pred4,pred6,pred7,pred8,suma,target,pred5,pred9,pred10,pred11
0,0,1,1,1,1,1,1,1,10,1,0,1,1,1
1,2,1,1,1,1,0,1,1,9,1,0,1,1,1
2,3,1,1,1,1,1,1,1,11,1,1,1,1,1
3,9,1,1,1,1,1,1,1,10,1,0,1,1,1
4,11,1,1,1,1,1,1,1,11,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,10861,1,1,1,1,1,1,1,9,1,1,0,0,1
3259,10865,1,1,1,1,1,1,1,11,1,1,1,1,1
3260,10868,1,1,1,1,1,1,1,11,1,1,1,1,1
3261,10874,1,1,1,1,1,1,1,11,1,1,1,1,1


In [110]:
submitFinal["suma"]=submitFinal.pred1+submitFinal.pred2+submitFinal.pred3+submitFinal.pred3\
  +submitFinal.pred4+submitFinal.pred5+submitFinal.pred6+submitFinal.pred7+submitFinal.pred8+submitFinal.pred9\
  +submitFinal.pred10

In [111]:
submitFinal["target"]=submitFinal.suma.apply(lambda x: 1 if x>4 else 0)

In [112]:
submitf = submitFinal[['id',"target"]]
submitf.to_csv('submitf.csv', index=False)
files.download('submitf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
submit3 = tw_test['id'].to_frame()
submit3['target'] = submitFinal["target_finaliti"]

In [ ]:
submit3

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


Ensamble

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [ ]:
eclf1  = VotingClassifier(estimators=[('m1', model1_1), ('m2', model2_1), ('m3', model3_1)], voting='hard')

In [ ]:
eclf1 = eclf1.fit([Xtrain, Xfeaturestrain],ytrain)

In [ ]:
clf1.predict([x_test, x_features_test])